In [60]:
#FN score he got was around 90% in macro, more then 80
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
import matplotlib.pyplot as plot

# we can use the LabelEncoder to encode the gender feature
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelBinarizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, GridSearchCV, cross_validate

# importing two different imputation methods that take into consideration all the features when predicting the missing values
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.dummy import DummyClassifier #Will identify the maority calss base line, model needs to do better then the baseline

# oversample the minority class using SMOTE
from imblearn.over_sampling import SMOTE, RandomOverSampler
from collections import Counter

# to reduce randomness then you put the seed
np.random.seed(42)

In [32]:
df = pd.read_csv("./data/student_prediction.csv")
df.rename(columns = {'KIDS':'PARENT_STATUS'}) #There is a column name error in the data noted in the Kaggle description, this fixes it.
df = df.drop(["STUDENTID"], axis=1)
print(f"Data shape: \n{df.shape}\n")
print(f"Data size: \n{df.size}\n")
print(f"Data ndim: \n{df.ndim}\n")
print("_____________________________________________\n")

data_train, data_test = train_test_split(df, test_size=0.2, random_state=42)
print(f"Train Data shape: \n{data_train}\n")
print(f"Test Data shape: \n{data_test}\n")


Data shape: 
(145, 32)

Data size: 
4640

Data ndim: 
2

_____________________________________________

Train Data shape: 
     AGE  GENDER  HS_TYPE  SCHOLARSHIP  WORK  ACTIVITY  PARTNER  SALARY  \
9      2       1        2            3     2         2        1       3   
4      2       2        1            3     2         2        1       3   
26     2       2        2            3     2         1        1       1   
120    2       1        1            3     1         1        1       2   
125    1       1        2            5     1         1        2       1   
..   ...     ...      ...          ...   ...       ...      ...     ...   
71     1       1        3            4     2         2        2       1   
106    1       2        2            4     2         1        2       1   
14     3       2        2            4     1         1        2       3   
92     1       2        2            3     2         2        2       1   
102    1       2        2            3     2        

In [33]:
df

,AGE,GENDER,HS_TYPE,SCHOLARSHIP,WORK,ACTIVITY,PARTNER,SALARY,TRANSPORT,LIVING,...,PREP_STUDY,PREP_EXAM,NOTES,LISTENS,LIKES_DISCUSS,CLASSROOM,CUML_GPA,EXP_GPA,COURSE ID,GRADE
0,2,2,3,3,1,2,2,1,1,1,...,1,1,3,2,1,2,1,1,1,1
1,2,2,3,3,1,2,2,1,1,1,...,1,1,3,2,3,2,2,3,1,1
2,2,2,2,3,2,2,2,2,4,2,...,1,1,2,2,1,1,2,2,1,1
3,1,1,1,3,1,2,1,2,1,2,...,1,2,3,2,2,1,3,2,1,1
4,2,2,1,3,2,2,1,3,1,4,...,2,1,2,2,2,1,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,2,1,2,3,1,1,2,1,1,2,...,1,1,2,1,2,1,3,3,9,5
141,1,1,2,4,2,2,2,1,4,2,...,1,1,3,2,2,1,5,3,9,5
142,1,1,1,4,2,2,2,1,1,1,...,1,1,3,3,2,1,4,3,9,1
143,2,1,2,4,1,1,1,5,2,3,...,2,1,2,1,2,1,5,3,9,4


In [34]:
#Extracting Labels

columns = data_train.columns.to_list()
columns_drop = columns.pop(-1)
labels_train = data_train.drop(columns, axis=1)
labels_test = data_test.drop(columns, axis=1)

print(f"labels_train: \n{labels_train}\n")
print(f"labels_test: \n{labels_test}\n")

features_train = data_train.drop(['GRADE'], axis=1)
features_test = data_test.drop(['GRADE'], axis=1)
print(f"features_train: \n{features_train }\n")
print(f"lfeatures_test: \n{features_test }\n")



labels_train: 
     GRADE
9        0
4        1
26       1
120      1
125      2
..     ...
71       6
106      7
14       2
92       7
102      7

[116 rows x 1 columns]

labels_test: 
     GRADE
69       5
140      5
27       1
19       3
42       1
117      1
126      3
108      6
84       7
18       2
12       0
55       3
128      0
78       4
73       6
36       2
112      2
133      3
100      6
101      7
94       6
136      0
11       0
66       5
31       3
45       3
51       4
76       4
111      2

features_train: 
     AGE  GENDER  HS_TYPE  SCHOLARSHIP  WORK  ACTIVITY  PARTNER  SALARY  \
9      2       1        2            3     2         2        1       3   
4      2       2        1            3     2         2        1       3   
26     2       2        2            3     2         1        1       1   
120    2       1        1            3     1         1        1       2   
125    1       1        2            5     1         1        2       1   
..   ...     ...

In [79]:

ohenc = OneHotEncoder(sparse=False)
ohenc_cols = ohenc.fit(features_train).get_feature_names_out()

data_prepared = pd.DataFrame(ohenc.fit_transform(features_train),columns=ohenc_cols, index=features_train.index)
#full_pipeline.fit_transform(data_train)
#pd.DataFrame(full_pipeline.fit_transform(train_df),columns=train_df.columns, index=train_df.index)
print(data_prepared)

     AGE_1  AGE_2  AGE_3  GENDER_1  GENDER_2  HS_TYPE_1  HS_TYPE_2  HS_TYPE_3  \
9      0.0    1.0    0.0       1.0       0.0        0.0        1.0        0.0   
4      0.0    1.0    0.0       0.0       1.0        1.0        0.0        0.0   
26     0.0    1.0    0.0       0.0       1.0        0.0        1.0        0.0   
120    0.0    1.0    0.0       1.0       0.0        1.0        0.0        0.0   
125    1.0    0.0    0.0       1.0       0.0        0.0        1.0        0.0   
..     ...    ...    ...       ...       ...        ...        ...        ...   
71     1.0    0.0    0.0       1.0       0.0        0.0        0.0        1.0   
106    1.0    0.0    0.0       0.0       1.0        0.0        1.0        0.0   
14     0.0    0.0    1.0       0.0       1.0        0.0        1.0        0.0   
92     1.0    0.0    0.0       0.0       1.0        0.0        1.0        0.0   
102    1.0    0.0    0.0       0.0       1.0        0.0        1.0        0.0   

     SCHOLARSHIP_1  SCHOLAR

In [80]:
# create the following four different models with their default hyperparameter values to be trained using the preprocessed data (0.5 * 4)
labelTrainFlat = labels_train.values.ravel()
# Gradient Boosting
gradientBoosting = GradientBoostingClassifier()
gradientBoosting = gradientBoosting.fit(data_prepared, labelTrainFlat)

# Decision Trees
decisionTree = DecisionTreeClassifier()
decisionTree = decisionTree.fit(data_prepared,labelTrainFlat)

# Random Forests
randomForest = RandomForestClassifier()
randomForest = randomForest.fit(data_prepared,labelTrainFlat)

In [120]:
parametersGradientBoosting = [
    {'min_samples_leaf': [0.00001, 1,2,4, 10],'min_samples_split': [2,3,4], 'n_estimators': [10,100,50, 120]}
]
scoringX = {"accuracy": "accuracy", "bal_accuracy": "balanced_accuracy", "F1_macro": "f1_macro"}

grid_searchGradientBoosting = GridSearchCV(gradientBoosting, parametersGradientBoosting, cv=3, scoring = scoringX, return_train_score=True, n_jobs=-1, refit='bal_accuracy')

grid_searchGradientBoosting.fit(data_prepared, labelTrainFlat)

print(f"Best parameters GradientBoosting: \n{grid_searchGradientBoosting.best_params_}\n")
print(f"Best estimator GradientBoosting: \n{grid_searchGradientBoosting.best_estimator_}\n")
print(f"Best score GradientBoosting: \n{grid_searchGradientBoosting.best_score_}\n")

Best parameters GradientBoosting: 
{'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 10}

Best estimator GradientBoosting: 
GradientBoostingClassifier(n_estimators=10)

Best score GradientBoosting: 
0.365530303030303



In [ ]:
parametersDecisionTree = [
    {'max_depth': [1,2,3,4], 'min_samples_leaf': [4,5,6], 'min_samples_split': [1,2,3]}
]

grid_searchDecisionTree = GridSearchCV(decisionTree, parametersDecisionTree, cv=4, scoring = scoringX, return_train_score=True, n_jobs=-1, refit='bal_accuracy')

grid_searchDecisionTree.fit(data_prepared, labelTrainFlat)

print(f"Best parameters DecisionTree: \n{grid_searchDecisionTree.best_params_}\n")
print(f"Best estimator DecisionTree: \n{grid_searchDecisionTree.best_estimator_}\n")
print(f"Best score DecisionTree: \n{grid_searchDecisionTree.best_score_}\n")

In [106]:
parametersRandomForest = [
    {'n_estimators': [145,150,155,190],'max_depth': [5,6,7], 'bootstrap': [True, False]}
]

grid_searchRandomForest = GridSearchCV(randomForest, parametersRandomForest, cv=4, scoring = scoringX, return_train_score=True, n_jobs=-1, refit='bal_accuracy')

grid_searchRandomForest.fit(data_prepared, labelTrainFlat)

print(f"Best parameters RandomForest: \n{grid_searchRandomForest.best_params_}\n")

print(f"Best estimator RandomForest: \n{grid_searchRandomForest.best_estimator_}\n")

print(f"Best score RandomForest: \n{grid_searchRandomForest.best_score_}\n")

Best parameters RandomForest: 
{'bootstrap': False, 'max_depth': 333, 'n_estimators': 155}

Best estimator RandomForest: 
RandomForestClassifier(bootstrap=False, max_depth=333, n_estimators=155)

Best score RandomForest: 
0.32161458333333337



In [121]:
# print the grid search cross-validation results listing the above mentioned evaluation methods (3)
cross_val_resultsGB = grid_searchGradientBoosting.cv_results_
cross_val_resultsDT = grid_searchDecisionTree.cv_results_
cross_val_resultsRF = grid_searchRandomForest.cv_results_



print(f"Mean Test Accuracy for GradientBoosting: \n{cross_val_resultsGB['mean_test_accuracy']}\n")
print(f"Balanced Test Accuracy for GradientBoosting: \n{cross_val_resultsGB['mean_test_bal_accuracy']}\n")
print(f"Mean F1 Macro for GradientBoosting: \n{cross_val_resultsGB['mean_test_F1_macro']}\n")

#DTC
print(f"Mean Test Accuracy for Decision Trees: \n{cross_val_resultsDT['mean_test_accuracy']}\n")
print(f"Balanced Test Accuracy for Decision Trees: \n{cross_val_resultsDT['mean_test_bal_accuracy']}\n")
print(f"Mean F1 Macro for Decision Trees: \n{cross_val_resultsDT['mean_test_F1_macro']}\n")

#RFC
print(f"Mean Test Accuracy for Random Forests: \n{cross_val_resultsRF['mean_test_accuracy']}\n")
print(f"Balanced Test Accuracy for Random Forests: \n{cross_val_resultsRF['mean_test_bal_accuracy']}\n")
print(f"Mean F1 Macro for Random Forests: \n{cross_val_resultsRF['mean_test_F1_macro']}\n")

#NB


Mean Test Accuracy for GradientBoosting: 
[0.42150247 0.35267656 0.36122357 0.33558255 0.38731444 0.34390463
 0.38708952 0.34390463 0.3699955  0.31848853 0.34435448 0.36167341
 0.4217274  0.35290148 0.3699955  0.33558255 0.39608637 0.33535762
 0.39608637 0.3699955  0.39586145 0.31848853 0.3699955  0.3097166
 0.37854251 0.35357625 0.3879892  0.35380117 0.37854251 0.35380117
 0.37089519 0.34502924 0.35290148 0.37089519 0.37112011 0.36234818
 0.35245164 0.37899235 0.31826361 0.35312641 0.36077373 0.31871345
 0.30994152 0.35335133 0.36954566 0.33625731 0.34412955 0.32726046
 0.34435448 0.34435448 0.36977058 0.34435448 0.34435448 0.34435448
 0.36977058 0.33580747 0.34435448 0.36144849 0.36977058 0.35290148]

Balanced Test Accuracy for GradientBoosting: 
[0.32564935 0.27624459 0.29237915 0.2508658  0.29945887 0.25995671
 0.30727814 0.26489899 0.27108586 0.2385101  0.26559343 0.28017677
 0.3655303  0.26614358 0.29090909 0.25440115 0.29646465 0.26450216
 0.31422258 0.2895202  0.29370491 0.2430

In [86]:
# use a dummy classifier to identify a simple baseline (i.e., a majority class baseline) so that you can compare your prediction results (3)
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(data_prepared, labelTrainFlat)
DummyClassifier(strategy='most_frequent')
print(f"Dummy Prediction: \n{dummy_clf.predict(data_prepared)}\n") 
print(f"Dummy Score: \n{dummy_clf.score(data_prepared, labelTrainFlat)}\n") 

Dummy Prediction: 
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1]

Dummy Score: 
0.27586206896551724



In [126]:

ohenc_cols_test = ohenc.fit(features_test).get_feature_names_out()
print(ohenc_cols_test)
data_prepared_test = pd.DataFrame(ohenc.fit_transform(features_test),columns=ohenc_cols_test, index=features_test.index)

# obtain predictions on test data using the best model from GridSearchCV (i.e., .best_estimator_) (2)
predictions_test = grid_searchGradientBoosting.best_estimator_.predict(data_prepared_test)

# generate the classification report and the confusion matrix for test predictions (3)
print(classification_report(labels_test.values.ravel(),predictions_test))
print(confusion_matrix(labels_test, predictions_test))

['AGE_1' 'AGE_2' 'AGE_3' 'GENDER_1' 'GENDER_2' 'HS_TYPE_1' 'HS_TYPE_2'
 'HS_TYPE_3' 'SCHOLARSHIP_2' 'SCHOLARSHIP_3' 'SCHOLARSHIP_4'
 'SCHOLARSHIP_5' 'WORK_1' 'WORK_2' 'ACTIVITY_1' 'ACTIVITY_2' 'PARTNER_1'
 'PARTNER_2' 'SALARY_1' 'SALARY_2' 'SALARY_3' 'SALARY_4' 'TRANSPORT_1'
 'TRANSPORT_2' 'TRANSPORT_3' 'TRANSPORT_4' 'LIVING_1' 'LIVING_2'
 'LIVING_3' 'MOTHER_EDU_1' 'MOTHER_EDU_2' 'MOTHER_EDU_3' 'MOTHER_EDU_4'
 'MOTHER_EDU_5' 'FATHER_EDU_1' 'FATHER_EDU_2' 'FATHER_EDU_3'
 'FATHER_EDU_4' 'FATHER_EDU_5' 'FATHER_EDU_6' '#_SIBLINGS_1'
 '#_SIBLINGS_2' '#_SIBLINGS_3' '#_SIBLINGS_4' '#_SIBLINGS_5' 'KIDS_1'
 'KIDS_2' 'KIDS_3' 'MOTHER_JOB_1' 'MOTHER_JOB_2' 'MOTHER_JOB_3'
 'MOTHER_JOB_4' 'FATHER_JOB_1' 'FATHER_JOB_2' 'FATHER_JOB_3'
 'FATHER_JOB_4' 'FATHER_JOB_5' 'STUDY_HRS_1' 'STUDY_HRS_2' 'STUDY_HRS_3'
 'STUDY_HRS_4' 'STUDY_HRS_5' 'READ_FREQ_1' 'READ_FREQ_2' 'READ_FREQ_3'
 'READ_FREQ_SCI_1' 'READ_FREQ_SCI_2' 'READ_FREQ_SCI_3' 'ATTEND_DEPT_1'
 'ATTEND_DEPT_2' 'IMPACT_1' 'IMPACT_2' 'IMPACT_3' 'ATTE

C:\Users\adamj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- FATHER_EDU_6
- MOTHER_EDU_5
- PREP_EXAM_3
- TRANSPORT_3
Feature names seen at fit time, yet now missing:
- CUML_GPA_1
- LIVING_4
- MOTHER_EDU_6
- MOTHER_JOB_5
- SALARY_5
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 110 features, but GradientBoostingClassifier is expecting 112 features as input.